# Lesson 7 - Schema Proposal for Unstructured Position Management Data

In this lesson, you will design agents that propose how to extract information from unstructured position management documents such as emails, chats, SOPs (Standard Operating Procedures), and SLA (Service Level Agreements).

You'll learn:
- how "named entity recognition" can identify trades, positions, accounts, and operational entities in unstructured text
- how facts can be extracted as a "triple" of subject, predicate and object from operational communications
- how to leverage unstructured data to enrich position management knowledge graphs

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>To access the helper.py, neo4j_for_adk.py and tools.py files :</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

</div>

## 7.1. Agents

<img src="images/entire_solution.png" width="500">

Two agents to propose data extraction from unstructured position management documents: a "named entity recognition" agent and a "fact extraction" agent:

- Input: `approved_user_goal` (position management requirements), `approved_files` (emails, chats, SOPs, SLAs), `approved_construction_plan` (position hierarchy schema)
- Output: 
    - `approved_entity_types` describing entities like Issues, Breaks, Escalations, Processes, Teams that could be extracted from operational documents
    - `approved_fact_types` describing how those entities relate to positions, accounts, and operational workflows
- Tools: `get_approved_user_goal`, `get_approved_files`, `get_well_known_types`, `sample_file`, `set_proposed_entities`, `get_proposed_entities`, `approve_proposed_entities`, `add_proposed_fact`, `get_proposed_facts`, `approve_proposed_facts`

**Workflow**

<img src="images/workflow.png" width="500">

Named entity recognition:

1. The context is initialized with an `approved_user_goal` (position management), `approved_files` (operational documents) and an `approved_construction_plan` (position hierarchy)
2. The agent analyzes unstructured text from emails, chats, SOPs, and SLAs, looking for relevant operational entity types 
3. The agent proposes a list of entity types (e.g., Issues, Breaks, Processes, Teams), seeking user approval

Fact extraction:

1. The context now includes `approved_entity_types`
2. The agent analyzes unstructured documents, looking for how those entities can be saved as fact triples
3. The agent proposes fact types connecting operational entities to position data, seeking user approval

## 7.2. Setup

The usual import of needed libraries, loading of environment variables, and connection to Neo4j.

In [ ]:
# Import necessary libraries

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.tools import ToolContext

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb, tool_success, tool_error

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

In [ ]:
# --- Define Model Constants for easier use ---
MODEL_GPT_4O = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT_4O)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, messages=[{"role": "user", "content": "Are you ready?"}], tools=[]))

print("\nOpenAI ready.")

In [ ]:
# Check connection to Neo4j by sending a query

neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")

print(neo4j_is_ready)

## 7.3. Named Entity Recognition (NER) Sub-agent

The NER agent is responsible for proposing entities that could be extracted from unstructured position management documents (emails, chats, SOPs, SLAs).

An entity is a person, place, thing, or operational concept that is relevant to position management operations.

There are two general kinds of entities:

1. Well-known entities: these closely correlate with nodes in the existing structured data
   - in position management, this would be things like Trades, Positions, Accounts, Books, LegalEntities, Instruments
2. Discovered entities: these are entities that are not pre-defined, but are mentioned in operational documents
    - in position management communications, this may be Issues, Breaks, Reconciliation Errors, Escalations, Processes, Teams, Systems, Controls

The general goal of the NER agent is to analyze the operational documents and propose entities that are 
relevant to the user goal of position tracking, break management, and operational risk analysis.

### 7.3.1. Agent Instructions (NER)


In [ ]:
ner_agent_role_and_goal = """
  You are a top-tier algorithm designed for analyzing operational documents from Global Position Management Operations 
  in capital markets and proposing the kind of named entities that could be extracted which would be relevant 
  for position tracking, break management, reconciliation, and operational risk analysis.
  """

In [ ]:
ner_agent_hints = """
  Entities are people, places, things and qualities, but not quantities. 
  Your goal is to propose a list of the type of entities, not the actual instances
  of entities.

  There are two general approaches to identifying types of entities:
  - well-known entities: these closely correlate with approved node labels in an existing position management graph schema
  - discovered entities: these may not exist in the graph schema, but appear consistently in operational documents

  **Capital Markets Context:**
  
  Common discovered entities in position management operations:
  - Issue: Problems, breaks, or errors in position data
  - ReconciliationBreak: Discrepancies between systems or with external parties
  - Escalation: Issues raised to higher management or support
  - Process: Operational workflows (e.g., "EOD Position Reconciliation", "Trade Allocation")
  - Team: Operational teams (e.g., "Position Control", "Middle Office", "Regulatory Reporting")
  - System: Trading or operations systems (e.g., "Summit", "Calypso", "Murex", "OMS")
  - Control: Operational controls or checks
  - Approval: Authorization workflows
  - Exception: Situations requiring manual intervention
  - Correction: Fixes applied to position or trade data
  - Owner: Responsible party for issues or processes

  Design rules for well-known entities:
  - always use existing well-known entity types from the position management schema
  - prefer reusing existing entity types (Trade, Position, Account, Book, Desk, LegalEntity, Instrument) rather than creating new ones
  
  Design rules for discovered entities:
  - discovered entities are consistently mentioned in operational documents and are highly relevant to position management operations
  - always look for entities that would provide more depth or breadth to the existing position graph
  - for example, if the graph has "Position" nodes, look through the text to discover entities like "ReconciliationBreak", "Issue", or "Process" that explain operational challenges
  - avoid quantitative types that may be better represented as a property on an existing entity or relationship
  - for example, do not propose "Amount" or "Quantity" as entity types. Those are better represented as properties on "Position" or "Trade"
  - focus on entities that support operational analysis: root cause of breaks, ownership of issues, process dependencies
"""

In [ ]:
ner_agent_chain_of_thought_directions = """
  Prepare for the task:
  - use the 'get_user_goal' tool to get the user goal
  - use the 'get_approved_files' tool to get the list of approved files
  - use the 'get_well_known_types' tool to get the approved node labels

  Think step by step:
  1. Sample some of the files using the 'sample_file' tool to understand the content
  2. Consider what well-known entities are mentioned in the text
  3. Discover entities that are frequently mentioned in the text that support the user's goal
  4. Use the 'set_proposed_entities' tool to save the list of well-known and discovered entity types
  5. Use the 'get_proposed_entities' tool to retrieve the proposed entities and present them to the user for their approval
  6. If the user approves, use the 'approve_proposed_entities' tool to finalize the entity types
  7. If the user does not approve, consider their feedback and iterate on the proposal
"""

In [ ]:
ner_agent_instruction = f"""
{ner_agent_role_and_goal}
{ner_agent_hints}
{ner_agent_chain_of_thought_directions}
"""

#print(ner_agent_instruction)

### 7.3.2. Tool Definitions (NER)

As in previous lessons, you'll define some tools that explictly follow
a propose then approve pattern. 

In [ ]:
# tools to propose and approve entity types
PROPOSED_ENTITIES = "proposed_entity_types"
APPROVED_ENTITIES = "approved_entity_types"

def set_proposed_entities(proposed_entity_types: list[str], tool_context:ToolContext) -> dict:
    """Sets the list proposed entity types to extract from unstructured text."""
    tool_context.state[PROPOSED_ENTITIES] = proposed_entity_types
    return tool_success(PROPOSED_ENTITIES, proposed_entity_types)

def get_proposed_entities(tool_context:ToolContext) -> dict:
    """Gets the list of proposed entity types to extract from unstructured text."""
    return tool_context.state.get(PROPOSED_ENTITIES, [])

def approve_proposed_entities(tool_context:ToolContext) -> dict:
    """Upon approval from user, records the proposed entity types as an approved list of entity types 

    Only call this tool if the user has explicitly approved the suggested files.
    """
    if PROPOSED_ENTITIES not in tool_context.state:
        return tool_error("No proposed entity types to approve. Please set proposed entities first, ask for user approval, then call this tool.")
    tool_context.state[APPROVED_ENTITIES] = tool_context.state.get(PROPOSED_ENTITIES)
    return tool_success(APPROVED_ENTITIES, tool_context.state[APPROVED_ENTITIES])

def get_approved_entities(tool_context:ToolContext) -> dict:
    """Get the approved list of entity types to extract from unstructured text."""
    return tool_context.state.get(APPROVED_ENTITIES, [])


The "well-known entities" are based on existing node labels used during graph construction.

This helper tool will get the existing node labels from the approved construction plan.

In [ ]:
def get_well_known_types(tool_context:ToolContext) -> dict:
    """Gets the approved labels that represent well-known entity types in the graph schema."""
    construction_plan = tool_context.state.get("approved_construction_plan", {})
    # approved labels are the keys for each construction plan entry where `construction_type` is "node"
    approved_labels = {entry["label"] for entry in construction_plan.values() if entry["construction_type"] == "node"}
    return tool_success("approved_labels", approved_labels)



The full toolset includes some existing tools that you'll import
plus the extra tools you just defined.

In [ ]:
from tools import get_approved_user_goal, get_approved_files, sample_file
ner_agent_tools = [
    get_approved_user_goal, get_approved_files, sample_file,
    get_well_known_types,
    set_proposed_entities,
    approve_proposed_entities
]

To see what the NER agent is working with, use the sample_file tool to look
at one of the markdown files.

In [ ]:
file_result = sample_file("emails.txt")

print(file_result["content"])

The operational documents contain emails, chats, SOPs, and SLAs from the Position Management Operations team that include:
- Reconciliation breaks and discrepancies
- Escalations to management or support teams
- Process descriptions and ownership
- System issues and dependencies
- Approval workflows and controls

For operational risk analysis and break management, you'll be interested in:
- Issues that cause position breaks or reconciliation failures
- Processes that are critical for accurate position reporting
- Teams responsible for various operational tasks
- Systems involved in the position management workflow

We won't provide explicit instructions about these operational concepts to the agent, instead relying
on a combination of the stated user goal along with instruction to find entity types
that would support position management operations.

### 7.3.3. Construct the Sub-agent (NER)

In [ ]:
NER_AGENT_NAME = "ner_schema_agent_v1"
ner_schema_agent = Agent(
    name=NER_AGENT_NAME,
    description="Proposes the kind of named entities related to position management operations that could be extracted from operational documents.",
    model=llm,
    instruction=ner_agent_instruction,
    tools=ner_agent_tools, 
)


The initial state is important in this phase, as the agent is designed to act
within a particular phase of an overall workflow.

The NER agent will need:

- the user goal, extended to mention operational documents and what operational entities to look for
- a list of unstructured files (emails, chats, SOPs, SLAs) that have been pre-approved
- the approved construction plan from the structured position management data design phase

In [ ]:
ner_agent_initial_state = {
    "approved_user_goal": {
        "kind_of_graph": "position hierarchy network",
        "description": """A comprehensive position management graph tracking trades, positions, accounts, books, desks, 
        and legal entities to support P&L attribution, risk analysis, and regulatory reporting (CCAR, Volcker, FRTB). 
        Enrich with operational documents (emails, chats, SOPs, SLAs) to identify reconciliation breaks, process dependencies, 
        team ownership, and system issues affecting position accuracy."""
    },
    "approved_files": [
        "emails.txt",
        "chats.txt",
        "sop.txt",
        "sla.txt"
    ],
    "approved_construction_plan": {
        "Trade": {
            "construction_type": "node",
            "label": "Trade",
        },
        "Position": {
            "construction_type": "node",
            "label": "Position",
        },
        "Account": {
            "construction_type": "node",
            "label": "Account",
        },
        "Book": {
            "construction_type": "node",
            "label": "Book",
        },
        "Desk": {
            "construction_type": "node",
            "label": "Desk",
        },
        "LegalEntity": {
            "construction_type": "node",
            "label": "LegalEntity",
        },
        "Instrument": {
            "construction_type": "node",
            "label": "Instrument",
        },
        "Counterparty": {
            "construction_type": "node",
            "label": "Counterparty",
        }
        # Relationship construction omitted, since it won't get used in this notebook
    }
}

OK, you're ready to run the agent. 

- use the make_agent_caller to create an execution environment
- prompt the agent with a single message that should kick-off the analysis
- expect the result to be a proposed list of entity types relevant to position management operations
- but *not* a list of approved entity types

**The entity types here may vary quite a bit. If you're not happy with the proposal,
you can run the cell again to get a new list. Look for entities like Issue, ReconciliationBreak, Process, Team, System, etc.**

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by LLMs can vary with each execution due to their stochastic nature. Your results might differ from those shown in the video.</p>

In [ ]:
from helper import make_agent_caller

ner_agent_caller = await make_agent_caller(ner_schema_agent, ner_agent_initial_state)

await ner_agent_caller.call("Add operational documents to the position management knowledge graph to identify breaks, issues, processes, and team ownership affecting position accuracy.")

# Alternatively, uncomment this line to get verbose output
# await ner_agent_caller.call("Add operational documents to identify position management issues.", True)

session_end = await ner_agent_caller.get_session()

print("\n---\n")

print("\nSession state: ", session_end.state)

if PROPOSED_ENTITIES in session_end.state:
    print("\nProposed entities: ", session_end.state[PROPOSED_ENTITIES])

if APPROVED_ENTITIES in session_end.state:
    print("\nInappropriately approved entities: ", session_end.state[APPROVED_ENTITIES])
else:
    print("\nAwaiting approval.")

**Note**

- the agent should identify operational entities like Issue, ReconciliationBreak, Process, Team, System, etc.
- if the agent confuses operational processes (like "Position Reconciliation Process") with the Position entity itself, that's expected
- the agent will see the term "Position" in the list of well-known entity types
- and it will notice mentions of positions in operational contexts
- to fix this, the schema proposal from the previous lesson could save descriptions for each node label to provide more context

Once you're happy with the proposal, you can tell the agent that you approve.

In [ ]:
await ner_agent_caller.call("Approve the proposed entities.")

session_end = await ner_agent_caller.get_session()

ner_end_state = session_end.state if session_end else {}

print("Session state: ", ner_end_state)

if APPROVED_ENTITIES in ner_end_state:
    print("\nApproved entities: ", ner_end_state[APPROVED_ENTITIES])
else:
    print("\nStill awaiting approval? That is weird. Please check the agent's state and the proposed entities.")

## 7.4. Fact Type Extraction Sub-agent


### 7.4.1. Agent Instructions (fact type extraction)

In [ ]:
fact_agent_role_and_goal = """
  You are a top-tier algorithm designed for analyzing operational documents from Global Position Management Operations
  and proposing the type of facts that could be extracted from text that would be relevant 
  for understanding position breaks, operational processes, team responsibilities, and system dependencies.
"""

In [ ]:
fact_agent_hints = """
  Do not propose specific individual facts, but instead propose the general type 
  of facts that would be relevant for position management operations. 
  For example, do not propose "Position 12345 has Issue ABC" but the general type of fact "Position has_issue Issue".
  
  Facts are triplets of (subject, predicate, object) where the subject and object are
  approved entity types, and the proposed predicate provides information about
  how they are related. 
  
  **Capital Markets Examples:**
  - (Position, has_issue, Issue)
  - (Account, contains, Position)
  - (Issue, assigned_to, Team)
  - (Process, depends_on, System)
  - (ReconciliationBreak, affects, Account)
  - (Team, owns, Process)
  - (Issue, escalated_to, Desk)
  - (System, generates, Position)

  Design rules for facts:
  - only use approved entity types as subjects or objects. Do not propose new types of entities
  - the proposed predicate should describe the relationship between the approved subject and object
  - the predicate should optimize for information that is relevant to position management operations:
    * Issue tracking and ownership
    * Process dependencies and workflows
    * System interactions
    * Team responsibilities
    * Reconciliation break analysis
  - the predicate must appear in the source text or be clearly implied. Do not guess.
  - use the 'add_proposed_fact' tool to record each proposed fact type
"""

In [ ]:
fact_agent_chain_of_thought_directions = """
    Prepare for the task:
    - use the 'get_approved_user_goal' tool to get the user goal
    - use the 'get_approved_files' tool to get the list of approved files
    - use the 'get_approved_entities' tool to get the list of approved entity types

    Think step by step:
    1. Use the 'get_approved_user_goal' tool to get the user goal
    2. Sample some of the approved files using the 'sample_file' tool to understand the content
    3. Consider how subjects and objects are related in the text
    4. Call the 'add_proposed_fact' tool for each type of fact you propose
    5. Use the 'get_proposed_facts' tool to retrieve all the proposed facts
    6. Present the proposed types of facts to the user, along with an explanation
"""

In [ ]:
fact_agent_instruction = f"""
{fact_agent_role_and_goal}
{fact_agent_hints}
{fact_agent_chain_of_thought_directions}
"""

### 7.4.2. Tool Definitions (fact type extraction)

In [ ]:
PROPOSED_FACTS = "proposed_fact_types"
APPROVED_FACTS = "approved_fact_types"

def add_proposed_fact(approved_subject_label:str,
                      proposed_predicate_label:str,
                      approved_object_label:str,
                      tool_context:ToolContext) -> dict:
    """Add a proposed type of fact that could be extracted from the files.

    A proposed fact type is a tuple of (subject, predicate, object) where
    the subject and object are approved entity types and the predicate 
    is a proposed relationship label.

    Args:
      approved_subject_label: approved label of the subject entity type
      proposed_predicate_label: label of the predicate
      approved_object_label: approved label of the object entity type
    """
    # Guard against invalid labels
    approved_entities = tool_context.state.get(APPROVED_ENTITIES, [])
    
    if approved_subject_label not in approved_entities:
        return tool_error(f"Approved subject label {approved_subject_label} not found. Try again.")
    if approved_object_label not in approved_entities:
        return tool_error(f"Approved object label {approved_object_label} not found. Try again.")
    
    current_predicates = tool_context.state.get(PROPOSED_FACTS, {})
    current_predicates[proposed_predicate_label] = {
        "subject_label": approved_subject_label,
        "predicate_label": proposed_predicate_label,
        "object_label": approved_object_label
    }
    tool_context.state[PROPOSED_FACTS] = current_predicates
    return tool_success(PROPOSED_FACTS, current_predicates)
    
def get_proposed_facts(tool_context:ToolContext) -> dict:
    """Get the proposed types of facts that could be extracted from the files."""
    return tool_context.state.get(PROPOSED_FACTS, {})


def approve_proposed_facts(tool_context:ToolContext) -> dict:
    """Upon user approval, records the proposed fact types as approved fact types

    Only call this tool if the user has explicitly approved the proposed fact types.
    """
    if PROPOSED_FACTS not in tool_context.state:
        return tool_error("No proposed fact types to approve. Please set proposed facts first, ask for user approval, then call this tool.")
    tool_context.state[APPROVED_FACTS] = tool_context.state.get(PROPOSED_FACTS)
    return tool_success(APPROVED_FACTS, tool_context.state[APPROVED_FACTS])


In [ ]:
fact_agent_tools = [
    get_approved_user_goal, get_approved_files, 
    get_approved_entities,
    sample_file,
    add_proposed_fact,
    get_proposed_facts,
    approve_proposed_facts
]

### 7.4.3. Construct the Sub-agent (fact type extraction)

In [ ]:
FACT_AGENT_NAME = "fact_type_extraction_agent_v1"
relevant_fact_agent = Agent(
    name=FACT_AGENT_NAME,
    description="Proposes the kind of relevant operational facts about position management that could be extracted from operational documents.",
    model=llm,
    instruction=fact_agent_instruction,
    tools=fact_agent_tools, 
)


<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by LLMs can vary with each execution due to their stochastic nature. Your results might differ from those shown in the video.</p>

In [ ]:
# make a copy of the NER agent's end state to use as the initial state for the fact agent
fact_agent_initial_state = ner_end_state.copy()

fact_agent_caller = await make_agent_caller(relevant_fact_agent, fact_agent_initial_state)

await fact_agent_caller.call("Propose fact types that can be found in the operational documents to connect issues, processes, teams, and systems to position data.")
# await fact_agent_caller.call("Propose fact types that can be found in the operational documents.", True)

session_end = await fact_agent_caller.get_session()

print("\n---\n")

print("\nSession state: ", session_end.state)

print("\nApproved entities: ", session_end.state.get(APPROVED_ENTITIES, []))

# Check that the agent proposed facts
if PROPOSED_FACTS in session_end.state:
    print("\nCorrectly proposed facts: ", session_end.state[PROPOSED_FACTS])
else:
    print("\nProposed facts not found in session state. What went wrong?")

# Check that the agent did not inappropriately approve facts
if APPROVED_FACTS in session_end.state:
    print("\nInappriately approved facts: ", session_end.state[APPROVED_FACTS])
else:
    print("\nApproved facts not found in session state, which is good.")

If you're happy with the fact type proposal, approve it.

In [ ]:
await fact_agent_caller.call("Approve the proposed fact types.")

session_end = await fact_agent_caller.get_session()

print("Session state: ", session_end.state)

if APPROVED_FACTS in session_end.state:
    print("\nApproved fact types: ", session_end.state[APPROVED_FACTS])
else:
    print("\nFailed to approve fact types.")